In [8]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gradio as gr
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize

In [10]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

In [11]:
import numpy as np
embeddings=np.load("embedding_matrix.npy")

In [15]:
def predict_fake_news(title, context):
    # Preprocess the inputs
    title = preprocess_text(title)
    context = preprocess_text(context)
    
    # Tokenize the inputs
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([title, context])
    
    title_seq = tokenizer.texts_to_sequences([title])
    context_seq = tokenizer.texts_to_sequences([context])
    
    # Define maximum lengths for padding
    max_len_title = 26
    max_len_context = 10074
    
    # Pad sequences
    padded_title_seq = pad_sequences(title_seq, maxlen=max_len_title, padding='post', truncating='post')
    padded_context_seq = pad_sequences(context_seq, maxlen=max_len_context, padding='post', truncating='post')
    
    # Convert to NumPy arrays
    inp1 = np.array(padded_title_seq)
    inp2 = np.array(padded_context_seq)
    
    # Load the pre-trained model
    model = tf.keras.models.load_model('model_arch_for_fake_news.h5')
    model.load_weights("model_for_fakenews_13.keras")
    
    # Make prediction
    prediction = model.predict([inp1, inp2])
    
    # Interpret the prediction
    if prediction[0][0] > 0.5:
        return "Fake News"
    else:
        return "Real News"

In [16]:
import gradio as gr

iface = gr.Interface(
    fn=predict_fake_news,
    inputs=[gr.Textbox(lines=2, placeholder="Enter the news title here..."), 
            gr.Textbox(lines=15, placeholder="Enter the news context here...")],
    outputs="text",
    title="Fake News Detector",
    description="Enter the news title and context to check if it's fake or real."
)

In [17]:
# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


C:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
